# Shift Cipher

Shift cipher is an encryption technique, where all characters of given text are shifted by a certain amount determined by a key, which is an integer value.

Assume a given text P needs to be encrypted using a key k, the cipher text C can be calculated as -
C = P + k

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters).

In [1]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    return result

Lets declare a plain text that we would need to encrypt.

In [4]:
P = 'This is a plain text.'

Encoding this string, we get -

In [5]:
T = encode(P)
print(T)

THISISAPLAINTEXT


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [6]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key matrix for the encryption by choosing a key at random from the key domain.

In [13]:
from random import choice
k_domain = [i for i in range(26)] # integers from 0 to 25 (included)
key = choice(k_domain)

In [14]:
print('Key - ', key)

Key -  13


In [22]:
def shiftcipher(text, cipher_key):
    cipher = ''
    
    for e in text:
        text_value = ord(e) - ord('A')
        cipher_value = (text_value + cipher_key) % 26
        cipher += chr(int(cipher_value + ord('A')))
    return cipher
    

def shiftdecipher(cipher_text, cipher_key):
    text = ''
    for e in cipher_text:
        cipher_value = ord(e) - ord('A')
        text_value = (cipher_value - cipher_key) % 26
        text += chr(int(text_value + ord('A')))
    return text

Now we test the cipher and decipher algorithms by encrypting and decrypting the text P

In [23]:
T = encode(P)
C = shiftcipher(T, key)
D = shiftdecipher(C, key)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key - {key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')

Given text - "This is a plain text."
Encoded - THISISAPLAINTEXT
Key - 13
Cipher text - GUVFVFNCYNVAGRKG
Decipher text - THISISAPLAINTEXT


## Test Against Builtin Cipher

Now, we can test the result against the built in Hill Cipher in sagemath.

In [26]:
A = ShiftCryptosystem(AlphabeticStrings())
E = A.encoding(encode(P))
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key -\b{key}')
C_test = A.enciphering(key, E)
D_test = A.deciphering(key, C_test)

# convert to python string
C_test = str(C_test)
D_test = str(D_test)

print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')

Text - This is a plain text.
Encoded - THISISAPLAINTEXT
Key -13
Cipher text - GUVFVFNCYNVAGRKG
Decipher text - THISISAPLAINTEXT


Comparing the built in cipher result with our implementation -

In [27]:
print('Results \t Implementation \t Built-in\n')
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in

Cipher Text 	 GUVFVFNCYNVAGRKG 	 GUVFVFNCYNVAGRKG
Decipher Text 	 THISISAPLAINTEXT 	 THISISAPLAINTEXT

Implementation is CORRECT


## Cryptoanalysis